In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
 
data = "All work and no play makes jack dull boy. All work and no play makes jack a dull boy."
stopWords = set(stopwords.words('english'))
words = word_tokenize(data)
wordsFiltered = [ w for w in words if w not in stopWords]
print(wordsFiltered)

['All', 'work', 'play', 'makes', 'jack', 'dull', 'boy', '.', 'All', 'work', 'play', 'makes', 'jack', 'dull', 'boy', '.']


In [2]:
def lex(statement):
    words = []
    for w in word_tokenize(statement):
        if w not in stopWords and w.isalpha():
            words.append(w)
    return words
    return [ w for w in word_tokenize(statement) if w not in stopWords]

In [3]:
lex(data)

['All',
 'work',
 'play',
 'makes',
 'jack',
 'dull',
 'boy',
 'All',
 'work',
 'play',
 'makes',
 'jack',
 'dull',
 'boy']

In [4]:
def parse(lexed_statement,ns=(1,2)):
    out = []
    for n in ns:
        for i in range(len(lexed_statement)):
            out.append(tuple(lexed_statement[i:i+n]))
    return set(tuple(out))

In [5]:
lexed  = lex(data)
parse(lexed)

{('All',),
 ('All', 'work'),
 ('boy',),
 ('boy', 'All'),
 ('dull',),
 ('dull', 'boy'),
 ('jack',),
 ('jack', 'dull'),
 ('makes',),
 ('makes', 'jack'),
 ('play',),
 ('play', 'makes'),
 ('work',),
 ('work', 'play')}

In [8]:
def lexed_and_parsed(statement):
    return parse(lex(statement))

In [17]:
truth_text_mapping = {
    'pants-fire':0,
    'false':0.1,
    'barely-true':0.25,
    'half-true':0.5,
    'mostly-true':0.75,
    'true':1
}
class Statement:
    def __init__(self, body, speaker, value):
        self.body = body
        self.speaker = speaker
        self.value = truth_text_mapping[value]
    
    
    @staticmethod
    def from_row(row):
        return Statement(value=row[1], body=row[2], speaker=row[4])
    
    
    def __repr__(self):
        arg_str =  str(', '.join(['='.join([i[0],repr(i[1])]) for i in vars(self).items()]))
        return "Statement({})".format(arg_str)
    
    
    def __str__(self):
        return repr(self)

In [18]:
import csv
statements = []
with open("../datasets/LIAR/train.tsv") as data_file:
    reader = csv.reader(data_file, delimiter='\t', quotechar='"')
    for row in reader:
        try:
            statements.append(Statement.from_row(row))
        except IndexError:
            print(row,len(row))
            
statements[:5]

[Statement(value=0.1, body='Says the Annies List political group supports third-trimester abortions on demand.', speaker='dwayne-bohac'),
 Statement(value=0.5, body='When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.', speaker='scott-surovell'),
 Statement(value=0.75, body='Hillary Clinton agrees with John McCain "by voting to give George Bush the benefit of the doubt on Iran."', speaker='barack-obama'),
 Statement(value=0.1, body='Health care reform legislation is likely to mandate free sex change surgeries.', speaker='blog-posting'),
 Statement(value=0.5, body='The economic turnaround started at the end of my term.', speaker='charlie-crist')]